In [ ]:
from pathlib import Path
import pandas as pd
from PIL import Image
from matplotlib.figure import Figure
import xlwings as xw
# Requires a license key: https://www.xlwings.org/trial
from xlwings.pro.reports import create_report

### Some data

In [ ]:
# Matplotlib
fig = Figure(figsize=(4, 3))
ax = fig.add_subplot(111)
ax.plot([1, 2, 3, 4, 5])

# Pandas DataFrame
perf_data = pd.DataFrame(index=['r1', 'r1'],
                         columns=['c0', 'c1'],
                         data=[[1., 2.], [3., 4.]])

# Picture
logo = Image.open(Path('xlwings.jpg'))

# Float
perf=0.12 * 100

# Collect all template vars (optional)
data = dict(perf_data=perf_data, logo=logo,
            perf=perf, fig=fig)

### Basic report creation

In [ ]:
wb = create_report('report_template.xlsx',
                   'report.xlsx',
                   **data)

### Handling App and Book

In [ ]:
wb = create_report('report_template.xlsx',
                   'report.xlsx',
                   **data,
                   app=xw.App(visible=False),
                   book_settings={'update_links': False})

# Clean up
wb.sheets.active['A1'].select()
wb.save()
wb.close()  # only quits app if visible=False